In [201]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_predict, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
%matplotlib inline

In [202]:
from sklearn.metrics import confusion_matrix

In [203]:
df = pd.read_csv("./DataFiles_2018/test_df_all.csv")

In [204]:
df.head()

,TeamID,OppTeamID,Won,years_coached,ConfAbbrev,conference_tourny_wins,conference_tourny_losses,num_of_sweet_16,num_of_elite_8,num_of_final_4,...,OppESR_mean,OppESR_min,OppRPI_max,OppRPI_mean,OppRPI_min,OppUSA_max,OppUSA_mean,OppUSA_min,Oppodds_to_win,Oppodds_to_final4
0,1104,1272,1,3,2,2.0,1.0,7.0,1.0,0.0,...,96.45,77,169,128.00,94,35.0,35.0,35.0,100000,100000
1,1233,1107,0,8,21,3.0,0.0,0.0,0.0,0.0,...,119.91,75,177,137.00,116,35.0,35.0,35.0,100000,100000
2,1112,1319,1,14,6,3.0,0.0,19.0,11.0,4.0,...,338.18,331,317,295.07,265,35.0,35.0,35.0,100000,100000
3,1113,1226,1,5,6,0.0,1.0,0.0,0.0,0.0,...,237.27,198,335,279.07,242,35.0,35.0,35.0,100000,100000
4,1116,1359,1,16,2,2.0,1.0,11.0,10.0,6.0,...,286.45,263,299,281.43,251,35.0,35.0,35.0,100000,100000


In [205]:
df.columns

Index(['TeamID', 'OppTeamID', 'Won', 'years_coached', 'ConfAbbrev',
       'conference_tourny_wins', 'conference_tourny_losses', 'num_of_sweet_16',
       'num_of_elite_8', 'num_of_final_4', 'num_of_top_2', 'num_of_title',
       'Ast', 'Blk', 'DR', 'FGA', 'FGA3', 'FGM', 'FGM3', 'FTA', 'FTM', 'OR',
       'PF', 'Score', 'Stl', 'TO', 'Win%', 'AP_max', 'AP_mean', 'AP_min',
       'EBP_max', 'EBP_mean', 'EBP_min', 'ESR_max', 'ESR_mean', 'ESR_min',
       'RPI_max', 'RPI_mean', 'RPI_min', 'USA_max', 'USA_mean', 'USA_min',
       'odds_to_win', 'odds_to_final4', 'Oppyears_coached', 'OppConfAbbrev',
       'Oppconference_tourny_wins', 'Oppconference_tourny_losses',
       'Oppnum_of_sweet_16', 'Oppnum_of_elite_8', 'Oppnum_of_final_4',
       'Oppnum_of_top_2', 'Oppnum_of_title', 'OppAst', 'OppBlk', 'OppDR',
       'OppFGA', 'OppFGA3', 'OppFGM', 'OppFGM3', 'OppFTA', 'OppFTM', 'OppOR',
       'OppPF', 'OppScore', 'OppStl', 'OppTO', 'OppWin%', 'OppAP_max',
       'OppAP_mean', 'OppAP_min', 'Opp

In [206]:
X= df.drop("Won", axis=1)
y = df["Won"]

In [207]:
 X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=33)

# Simple Logistic Regression

In [208]:
lr = LogisticRegression()
ss_lr = StandardScaler()

In [209]:
X_train_ss_lr = ss_lr.fit_transform(X_train)
X_holdout_ss_lr = ss_lr.transform(X_holdout)
lr.fit(X_train_ss_lr,y_train)
lr.score(X_train_ss_lr,y_train)

0.7686440677966102

0.7621235941468134

In [210]:
coef = pd.DataFrame(lr.coef_).T
col = pd.DataFrame(X_train.columns)
feat_lr = pd.concat([col,coef], axis=1)
feat_lr.columns = ["features", "importance"]
feat_lr["abs_import"] = abs(feat_lr["importance"])

print(feat_lr.sort_values("abs_import", ascending=False).to_string())

                       features  importance  abs_import
3                    ConfAbbrev   -1.087314    1.087314
44                OppConfAbbrev    1.073787    1.073787
25                         Win%    0.614065    0.614065
33                     ESR_mean   -0.595978    0.595978
66                      OppWin%   -0.488749    0.488749
71                  OppEBP_mean   -0.338860    0.338860
70                   OppEBP_max    0.330730    0.330730
69                    OppAP_min   -0.314135    0.314135
80                  OppUSA_mean    0.310779    0.310779
49            Oppnum_of_final_4   -0.303640    0.303640
30                     EBP_mean   -0.298810    0.298810
31                      EBP_min    0.282379    0.282379
68                   OppAP_mean   -0.267598    0.267598
32                      ESR_max    0.258815    0.258815
77                  OppRPI_mean    0.256432    0.256432
67                    OppAP_max    0.245088    0.245088
7                num_of_elite_8    0.234531    0

In [211]:
lr.score(X_holdout_ss_lr,y_holdout)

0.7703389830508475

# Simple KNN

In [212]:
knn = KNeighborsClassifier(n_neighbors=10,weights="distance")
ss_knn = StandardScaler()

In [213]:
X_train_ss_knn = ss_knn.fit_transform(X_train)
X_holdout_ss_knn = ss_knn.transform(X_holdout)
knn.fit(X_train_ss_knn,y_train)
knn.score(X_train_ss_knn,y_train)

0.9093220338983051

0.9118394001693071

In [214]:
knn.score(X_holdout_ss_knn,y_holdout)

0.7813559322033898

0.6840768951759159

In [215]:
cross_val_score(knn, X_train,y_train, cv=5)

array([0.77291961, 0.74293785, 0.76271186, 0.72033898, 0.7553041 ])

In [216]:
pred_knn = knn.predict(X)
cm = confusion_matrix(y, pred_knn)
cm

array([[1746,  614],
       [ 555, 1805]])

array([[4568,  945],
       [4044, 1469]])

# Simple DecisionTree

In [217]:
tree = DecisionTreeClassifier(max_depth=6, criterion="entropy")
tree.fit(X_train,y_train)
cross_val_score(tree,X_holdout,y_holdout,cv=3)

array([0.74111675, 0.70304569, 0.67091837])

# GridSearch LR

In [218]:
lr = LogisticRegression()
ss = StandardScaler

In [219]:
X_train_ss_lr = ss_lr.fit_transform(X_train)
X_holdout_ss_lr = ss_lr.transform(X_holdout)
params = {
    "C" : [1.0,2.0,3.0,4.0],
    "tol" : [.0001, .001, .0005]
}
lr_gs = GridSearchCV(lr,params)
lr_gs.fit(X_train_ss_lr,y_train)
print(lr_gs.best_params_)
lr_gs.best_score_

{'C': 1.0, 'tol': 0.001}


0.7502824858757062

0.7563187809892369

In [220]:
lr_gs.score(X_holdout_ss_lr,y_holdout)

0.7703389830508475

# GridSearch KNN

In [221]:
knn = KNeighborsClassifier()
ss_knn = StandardScaler()

In [222]:
X_train_ss_knn = ss_knn.fit_transform(X_train)
X_holdout_ss_knn = ss_knn.transform(X_holdout)
params = {
    "n_neighbors" : [35,40,50,55,60],
    "weights" : ["uniform", "distance"],
    "p" : [1,2]
}
knn_gs = GridSearchCV(knn,params)
knn_gs.fit(X_train_ss_knn,y_train)
print(knn_gs.best_params_)
knn_gs.best_score_

{'n_neighbors': 40, 'p': 2, 'weights': 'distance'}


0.7649717514124293

0.7322529931067844

In [223]:
knn_gs.score(X_holdout_ss_knn,y_holdout)

0.7966101694915254

In [224]:
pred_knn_gs = knn_gs.predict(X)
cm = confusion_matrix(y, pred_knn_gs)
cm

array([[1815,  545],
       [ 593, 1767]])

array([[4769,  744],
       [2346, 3167]])

# GridSearch Decision Tree

In [225]:
tree = DecisionTreeClassifier()
params = {
    "max_depth" : [None,5,8],
    "criterion" : ["gini", "entropy"],
    "max_features" : [None,10,.5,"sqrt"]
}
tree_gs = GridSearchCV(tree,params)
tree_gs.fit(X_train,y_train)
print(tree_gs.best_params_)

tree_gs.best_score_

{'criterion': 'gini', 'max_depth': 5, 'max_features': None}


0.7406779661016949

0.7292296529205466

In [226]:
lr_gs.score(X_holdout,y_holdout)

0.6228813559322034

0.5937179924889041

# Boosting

In [227]:
ada = AdaBoostClassifier(random_state=33)
ada_params = {
    "base_estimator" : [LogisticRegression()],
    "n_estimators" : [30,40,50,60],
    "learning_rate" : [1.0, 2.0, .5]
}
gs_ada = GridSearchCV(ada,ada_params)
gs_ada.fit(X_train, y_train)
print(gs_ada.best_score_)
gs_ada.best_params_

0.7494350282485875


{'base_estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'learning_rate': 0.5,
 'n_estimators': 50}

0.7557141129519893

In [228]:
pred_ada_bc = gs_ada.predict(X)
cm = confusion_matrix(y, pred_ada_bc)
cm

array([[1814,  546],
       [ 555, 1805]])

array([[4191, 1322],
       [1342, 4171]])

In [229]:
bc = BaggingClassifier(random_state=33)
bc_params = {
    "base_estimator" : [LogisticRegression(),KNeighborsClassifier(n_neighbors= 55, p= 2, weights= 'distance')],
    "n_estimators" : [10,20,30,35],
    "max_features" : [.5, .75, 1]
}
gs_bc = GridSearchCV(bc, bc_params)
gs_bc.fit(X_train, y_train)
print(gs_bc.best_score_)
gs_bc.best_params_

0.7624293785310734


{'base_estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'max_features': 0.5,
 'n_estimators': 10}

.7581327851009796

In [230]:
pred_gs_bc = gs_bc.predict(X)
cm = confusion_matrix(y, pred_gs_bc)
cm

array([[1800,  560],
       [ 566, 1794]])

array([[4164, 1349],
       [1321, 4192]])

# Nueral Networks

In [231]:

ss = StandardScaler()

In [232]:
X_train_ss = ss.fit_transform(X_train)
X_holdout_ss = ss.transform(X_holdout)

model = Sequential()
model.add(Dense(80, input_dim=X.shape[1], activation="relu"))
model.add(Dense(40, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
# ^^ always must have sigmoid for a binary classification problem ^^
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_ss, y_train, validation_data=(X_holdout_ss, y_holdout), epochs=30)

Train on 3540 samples, validate on 1180 samples
Epoch 1/30
3540/3540 [==============================] - 1s 237us/step - loss: 0.5192 - acc: 0.7339 - val_loss: 0.4783 - val_acc: 0.7610
Epoch 2/30
3540/3540 [==============================] - 0s 58us/step - loss: 0.4709 - acc: 0.7641 - val_loss: 0.4616 - val_acc: 0.7788
Epoch 3/30
3540/3540 [==============================] - 0s 58us/step - loss: 0.4559 - acc: 0.7788 - val_loss: 0.4572 - val_acc: 0.7720
Epoch 4/30
3540/3540 [==============================] - 0s 53us/step - loss: 0.4432 - acc: 0.7864 - val_loss: 0.4620 - val_acc: 0.7737
Epoch 5/30
3540/3540 [==============================] - 0s 54us/step - loss: 0.4342 - acc: 0.7881 - val_loss: 0.4542 - val_acc: 0.7797
Epoch 6/30
3540/3540 [==============================] - 0s 56us/step - loss: 0.4233 - acc: 0.7955 - val_loss: 0.4584 - val_acc: 0.7839
Epoch 7/30
3540/3540 [==============================] - 0s 57us/step - loss: 0.4163 - acc: 0.7997 - val_loss: 0.4578 - val_acc: 0.7932
Epoch 

8787/8787 [==============================] - 1s 58us/step - loss: 0.3222 - acc: 0.8447 - val_loss: 0.6929 - val_acc: 0.7054

# Predictions
- We are allowed two predictions for the Kaggle Compition. I will be choosing two models from above to predict with. 

In [233]:
test = pd.read_csv("./DataFiles_2018/predict_on.csv")

In [235]:
sample = pd.read_csv("./DataFiles_2018/SampleSubmissionStage2.csv")

In [238]:
sample_gs_bc = sample.copy()
sample_lr_gs = sample.copy()

In [240]:
pred_gs_bc = gs_bc.predict_proba(test)[:,1]

In [241]:
sample_gs_bc["Pred"] = pred_gs_bc
sample_gs_bc.to_csv("Sub_gs_bc.csv", index=False)

In [245]:
pred_lr_gs = lr_gs.predict_proba(test)

/Users/chrisjohnson/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [246]:
sample_lr_gs["Pred"] = pred_lr_gs
sample_lr_gs.to_csv("sub_lr_gs.csv", index=False)